# Sentiment Analysis

In [1]:

import pandas as pd
import numpy as np
import sqlite3
import itertools
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MSend\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
conn = sqlite3.connect('../datasets/lyrics.db')
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
sql_df = pd.read_sql_query("SELECT * FROM lyrics", conn)
#sql_df = sql_df[sql_df.vader_neg.isnull()]
# see if '\n' is in the lyrics
#sql_df = sql_df[sql_df.lyric.str.contains('\n')]
len(sql_df)

7166

In [10]:
#if we want to remove the song name and lyric from beginning, does not work for all, scared to use and mess up lyric data
for row in sql_df[2:3].iterrows():
    uri = row[1]['uri']
    song = row[1]['song'][1:-1]
    lyr = row[1]['lyric'].split(song+'Lyrics')[-1][1:]
    print(lyr)
    #query = "UPDATE lyrics SET polarity = {polar}, subjectivity = {subject}, vader = \'{vader}\', sentiment = \'{sentiment}\' WHERE uri = \'{uri}\'".format(polar = polarity, subject = subjecitvy, uri = uri, vader = vader, sentiment = sentiment)
    #conn.execute(query)
    #conn.commit()

4+35 Lyrics Hmm You might think Im crazy The way Ive been cravin If I put it quite plainly Just gimme them babies So what you doin tonight? Better say, Doin you right Yeah Watchin movies, but we aint seen a thing tonight Yeah I dont wanna keep you up You up But show me, can you keep it up? It up Cause then Ill have to keep you up Shit, maybe Ima keep you up, boy Ive been drinkin coffee Ive been drinkin coffee; coffee And Ive been eatin healthy Ive been eatin healthy; healthy You know I keep it squeaky, yeah You know I keep it squeaky Savin up my energy Yeah, yeah, savin up my energy Can you stay up all night? Fuck me til the daylight Thirty-four, thirty-five Yeah, yeah, yeah, yeah Can you stay up all night? All night Fuck mе til the daylight Thirty-four, thirty-five Yeah, yеah, yeah, yeah You drink it just like water Water You say it tastes like candy So what you doin tonight? Tonight Better say, Doin you right Alright Watchin movies, but we aint seen a thing tonight Yeah I dont wanna 

In [78]:
for row in sql_df.iterrows():
    uri = row[1]['uri']
    doc = nlp(row[1]['lyric'])
    polarity = doc._.blob.polarity     
    subjecitvy = doc._.blob.subjectivity 
    vader_info = analyzer.polarity_scores(row[1]['lyric'])
    vader = vader_info['compound']
    negativity = vader_info['neg']
    positivity = vader_info['pos']
    neutrality = vader_info['neu']
    sentiment = 1 if vader >= 0.5 else -1 if vader <=0.5 else 0
    #get sentiment as 1 or -1 either with polarity or subjectivity or both

    query = "UPDATE lyrics SET polarity = {polar}, subjectivity = {subject}, vader = \'{vader}\', sentiment = \'{sentiment}\' , vader_neg = \'{neg}\', vader_pos = \'{pos}\', vader_neut =\'{neut}\' WHERE uri = \'{uri}\'".format(polar = polarity,
                subject = subjecitvy, uri = uri, vader = vader, 
                sentiment = sentiment,pos = positivity,neg = negativity, neut = neutrality)
    conn.execute(query)
    conn.commit()